In [1]:
# Import the 'io' module, which provides core tools for working with streams of data.
import io

# Import the 'pandas' library and alias it as 'pd' for easier usage.
import pandas as pd

# Import the 'requests' library, which allows making HTTP requests.
import requests

In [2]:
# Store the URL of the CSV file containing Uber data in a variable.
url = 'https://storage.googleapis.com/uber-data-engineering-project/uber_data.csv'

# Use the 'requests' library to send an HTTP GET request to the specified URL and store the response.
response = requests.get(url)

In [3]:
# Use the 'pd.read_csv' function from the 'pandas' library to read the data from the HTTP response.
# 'io.StringIO(response.text)' converts the response content to a stream-like object for reading.
# The 'sep' parameter specifies the delimiter used in the CSV file.

df = pd.read_csv(io.StringIO(response.text), sep=',')

In [4]:
# Convert the 'tpep_pickup_datetime' column in the DataFrame to datetime format.
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# Convert the 'tpep_dropoff_datetime' column in the DataFrame to datetime format.
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [5]:
# Remove duplicate rows from the DataFrame and reset the index.
df = df.drop_duplicates().reset_index(drop=True)

# Create a new column 'trip_id' in the DataFrame with unique identifiers based on the index.
df['trip_id'] = df.index

# Display the first few rows of the DataFrame 'df' to inspect its initial data and structure.
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_id
0,1,2016-03-01,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35,0
1,1,2016-03-01,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35,1
2,2,2016-03-01,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80,2
3,2,2016-03-01,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,3
4,2,2016-03-01,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80,4


## Creating the Dimension Tables

### 1. datetime_dim

In [6]:
# Create a new DataFrame 'datetime_dim' to extract and transform date and time components from the original DataFrame 'df'.

# Extract 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' columns and reset the index.
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)

# Extract hour, day, month, year, and weekday components from 'tpep_pickup_datetime'.
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

# Extract hour, day, month, year, and weekday components from 'tpep_dropoff_datetime'.
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

# Assign unique 'datetime_id' based on the index.
datetime_dim['datetime_id'] = datetime_dim.index

# Reorder and select specific columns in 'datetime_dim'.
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

# Display the head (first few rows) of the transformed 'datetime_dim' DataFrame.
datetime_dim.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2016-03-01,0,1,3,2016,1,2016-03-01 00:07:55,0,1,3,2016,1
1,1,2016-03-01,0,1,3,2016,1,2016-03-01 00:11:06,0,1,3,2016,1
2,2,2016-03-01,0,1,3,2016,1,2016-03-01 00:31:06,0,1,3,2016,1
3,3,2016-03-01,0,1,3,2016,1,2016-03-01 00:00:00,0,1,3,2016,1
4,4,2016-03-01,0,1,3,2016,1,2016-03-01 00:00:00,0,1,3,2016,1


### 2. passenger_count_dim

In [7]:
# Create a new DataFrame 'passenger_count_dim' to isolate the 'passenger_count' column from the original DataFrame 'df'.
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)

# Assign unique 'passenger_count_id' based on the index.
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index

# Reorder and select specific columns in 'passenger_count_dim'.
passenger_count_dim = passenger_count_dim[['passenger_count_id', 'passenger_count']]



### 3. trip_distance_dim

In [8]:
# Create a new DataFrame 'trip_distance_dim' to isolate the 'trip_distance' column from the original DataFrame 'df'.
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)

# Assign unique 'trip_distance_id' based on the index.
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index

# Reorder and select specific columns in 'trip_distance_dim'.
trip_distance_dim = trip_distance_dim[['trip_distance_id', 'trip_distance']]

### 4. rate_code_dim

In [9]:
# Define a dictionary 'rate_code_type' that maps RatecodeID values to their corresponding rate code names.
rate_code_type = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride"
}

# Create a new DataFrame 'rate_code_dim' to isolate the 'RatecodeID' column from the original DataFrame 'df'.
rate_code_dim = df[['RatecodeID']].reset_index(drop=True)

# Assign unique 'rate_code_id' based on the index.
rate_code_dim['rate_code_id'] = rate_code_dim.index

# Map 'RatecodeID' values to their corresponding rate code names using the defined 'rate_code_type' dictionary.
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)

# Reorder and select specific columns in 'rate_code_dim'.
rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]

# Display the first few rows of the 'rate_code_dim' DataFrame to inspect the transformed rate code data.
rate_code_dim.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,1,Standard rate
2,2,1,Standard rate
3,3,1,Standard rate
4,4,3,Newark


### 5. pickup_location_dim

In [10]:
# Create a new DataFrame 'pickup_location_dim' to isolate the pickup longitude and latitude columns from the original DataFrame 'df'.
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)

# Assign unique 'pickup_location_id' based on the index.
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index

# Reorder and select specific columns in 'pickup_location_dim'.
pickup_location_dim = pickup_location_dim[['pickup_location_id', 'pickup_latitude', 'pickup_longitude']]

### 6. dropoff_location_dim

In [11]:
# Create a new DataFrame 'dropoff_location_dim' to isolate the dropoff longitude and latitude columns from the original DataFrame 'df'.
dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)

# Assign unique 'dropoff_location_id' based on the index.
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index

# Reorder and select specific columns in 'dropoff_location_dim'.
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id', 'dropoff_latitude', 'dropoff_longitude']]

### 7. payment_type_dim

In [12]:
# Define a dictionary 'payment_type_name' that maps payment_type values to their corresponding payment type names.
payment_type_name = {
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}

# Create a new DataFrame 'payment_type_dim' to isolate the 'payment_type' column from the original DataFrame 'df'.
payment_type_dim = df[['payment_type']].reset_index(drop=True)

# Assign unique 'payment_type_id' based on the index.
payment_type_dim['payment_type_id'] = payment_type_dim.index

# Map 'payment_type' values to their corresponding payment type names using the defined 'payment_type_name' dictionary.
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)

# Reorder and select specific columns in 'payment_type_dim'.
payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]

# Display the column names of the 'payment_type_dim' DataFrame to inspect its structure.
payment_type_dim.columns

Index(['payment_type_id', 'payment_type', 'payment_type_name'], dtype='object')

### Fact Table

In [13]:
# Create the fact table by merging multiple dimension tables with the original DataFrame 'df'.

# Merge 'passenger_count_dim' with 'df' using 'trip_id' as the common key.
# Repeat this process for other dimension tables.
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id') \
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

# Display the column names of the 'fact_table' DataFrame to inspect its structure.
fact_table.columns

# Display the 'fact_table' DataFrame to inspect the final consolidated data for analysis.
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,1,1,1,N,1,1,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,2,2,2,2,N,2,2,2,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,3,2,3,3,3,3,N,3,3,3,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,4,2,4,4,4,4,N,4,4,4,98.0,0.0,0.0,0.00,15.50,0.3,113.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,99995,99995,99995,99995,N,99995,99995,99995,5.0,0.0,0.5,0.00,0.00,0.3,5.80
99996,99996,1,99996,99996,99996,99996,N,99996,99996,99996,14.0,0.0,0.5,2.00,0.00,0.3,16.80
99997,99997,1,99997,99997,99997,99997,N,99997,99997,99997,29.0,0.0,0.5,8.80,5.54,0.3,44.14
99998,99998,2,99998,99998,99998,99998,N,99998,99998,99998,5.5,0.5,0.5,1.36,0.00,0.3,8.16
